#Cuestiones generales

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Train set

Descargo y leo los csv

In [3]:
id = '1VvZp2MML5dLce79TMsrCQm5VkvQn93zN'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_identity.csv')

train_identity = pd.read_csv('train_identity.csv')

In [4]:
id = '18ZAaojp_783CfnqhNj43w1vgwZOH221D'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('train_transaction.csv')

train_transaction = pd.read_csv('train_transaction.csv')

Mergeo los dos datasets que tengo

In [5]:
df = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [6]:
df.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,debit,272.0,87.0,36.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,debit,126.0,87.0,0.0,NaN,yahoo.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,mail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,debit,337.0,87.0,NaN,NaN,anonymous.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,debit,204.0,87.0,19.0,NaN,yahoo.com,NaN,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,2.0,61.0,61.0,30.0,318.0,30.0

Limpio la RAM

In [7]:
train_transaction = None
train_identity = None
downloaded = None

#Validation set

Dejo un 30% del dataset original como validation set

In [8]:
len(df.index)

590540

In [9]:
validation = df.loc[0:(0.3 * len(df.index)), :]

In [10]:
len(validation.index)

177163

In [11]:
train = df.loc[(0.3 * len(df.index)):len(df.index), :]

In [12]:
len(train.index)

413378

In [13]:
train.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
177162,3164162,0,3864166,39.00,W,10447,490.0,150.0,visa,226.0,debit,315.0,87.0,NaN,NaN,yahoo.com,NaN,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,4.0,1.0,182.0,182.0,6.0,182.0,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177163,3164163,1,3864195,49.00,W,1802,555.0,150.0,visa,226.0,debit,170.0,87.0,NaN,NaN,gmail.com,NaN,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,4.0,0.0,3.0,1.0,3.0,3.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177164,3164164,0,3864205,102.00,W,2114,396.0,150.0,mastercard,224.0,debit,315.0,87.0,NaN,NaN,NaN,NaN,13.0,20.0,0.0,0.0,1.0,6.0,0.0,0.0,9.0,0.0,9.0,0.0,53.0,10.0,6.0,6.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177165,3164165,1,3864212,24.00,W,10057,225.0,150.0,mastercard,224.0,debit,269.0,87.0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177166,3164166,0,3864218,422.50,W,16659,170.0,150.0,visa,226.0,credit,264.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177167,3164167,0,3864222,25.95,W,13052,254.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,gmail.com,NaN,4.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,3.0,0.0,22.0,4.0,121.0,121.0,12.0,158.0,12.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177168,3164168,0,3864228,221.99,W,13864,561.0,150.0,visa,226.0,debit,123.0,87.0,NaN,NaN,gmail.com,NaN,2.0,4.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,48.0,48.0,48.0,48.0,48.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177169,3164169,0,3864242,35.95,W,4501,390.0,150.0,mastercard,224.0,debit,498.0,87.0,NaN,NaN,gmail.com,NaN,83.0,67.0,0.0,0.0,188.0,50.0,0.0,0.0,58.0,0.0,63.0,0.0,488.0,68.0,102.0,102.0,7.0,338.0,21.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177170,3164170,0,3864249,117.00,W,9500,321.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,0.0,NaN,NaN,282.0,282.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177171,3164171,0,3864277,209.95,W,7826,481.0,150.0,mastercard,224.0,debit,494.0,87.0,4.0,NaN,NaN,NaN,154.0,135.0,0.0,0.0,145.0,105.0,0.0,0.0,92.0,0.0,106.0,0.0,477.0,123.0,53.0,53.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [14]:
validation.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
5,2987005,0,86510,49.0,W,5937,555.0,150.0,visa,226.0,debit,272.0,87.0,36.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2987006,0,86522,159.0,W,12308,360.0,150.0,visa,166.0,debit,126.0,87.0,0.0,NaN,yahoo.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2987007,0,86529,422.5,W,12695,490.0,150.0,visa,226.0,debit,325.0,87.0,NaN,NaN,mail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2987008,0,86535,15.0,H,2803,100.0,150.0,visa,226.0,debit,337.0,87.0,NaN,NaN,anonymous.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
9,2987009,0,86536,117.0,W,17399,111.0,150.0,mastercard,224.0,debit,204.0,87.0,19.0,NaN,yahoo.com,NaN,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,1.0,0.0,12.0,2.0,61.0,61.0,30.0,318.0,30.0

In [15]:
columnas_train = train.columns

Limpio la RAM

In [16]:
df = None

#Encoding train y validation set

In [17]:
from sklearn.preprocessing import OneHotEncoder

Diccionario en el que guardo como claves las columnas y como valores una lista de la forma [encoder, mean]

In [18]:
dic = {}

In [19]:
for c in train.columns:
  if c == 'isFraud': #a la columna isFraud no la toco
      continue

  if c == 'id_31': #aplico CountVectorizer a la columna id_30
      from sklearn.feature_extraction.text import CountVectorizer
      vectorizer = CountVectorizer(max_features=7)
      train[c].fillna(' ', inplace=True)
      vectorizer.fit(train.id_31)

      feature = vectorizer.transform(train.id_31).toarray()
      new_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names(), index=train.index)
      train = pd.concat([train, new_df], axis=1).drop(columns=[c])

      validation[c].fillna(' ', inplace=True)
      feature = vectorizer.transform(validation.id_31).toarray()
      new_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names(), index=validation.index)
      validation = pd.concat([validation, new_df], axis=1).drop(columns=[c])

      dic[c] = [vectorizer, None]

      vectorizer = None
      feature = None
      feature_labels = None
      new_df = None

      continue


  if c == 'DeviceType': #aplico OneHotEncoding a las columnas ProductCD y card4
      ohe = OneHotEncoder()
      ohe.fit(train[[c]])

      feature = ohe.transform(train[[c]]).toarray()
      new_df = pd.DataFrame(feature, columns=ohe.categories_, index=train.index)
      train = pd.concat([train, new_df], axis=1).drop(columns=[c])

      feature = ohe.transform(validation[[c]]).toarray()
      new_df = pd.DataFrame(feature, columns=ohe.categories_, index=validation.index)
      validation = pd.concat([validation, new_df], axis=1).drop(columns=[c])

      dic[c] = [ohe, None]

      ohe = None
      feature = None
      feature_labels = None
      new_df = None

      continue

  if (train[c].dtype.kind not in 'biufc'): #aplico MeanEncoding al resto de las columnas no numéricas
      mean_encoded_subject = train.groupby(c)['isFraud'].mean().to_dict()

      train[c] =  train[c].map(mean_encoded_subject)
      validation[c] = validation[c].map(mean_encoded_subject)

      dic[c] = [mean_encoded_subject, train[c].mean()]

      mean_encoded_subject = None

  aux = train[c].mean()
  train[c].fillna(aux, inplace=True)
  validation[c].fillna(aux, inplace=True)
  if c not in dic:
    dic[c] = [None, aux]

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [20]:
train.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceInfo,65,android,chrome,for,generic,mobile,safari,"(desktop,)","(mobile,)","(nan,)"
177162,3164162,0,3864166,39.00,0.021053,10447,490.0,150.0,0.038016,226.0,0.025510,315.0,87.0,116.06686,226.707442,0.025593,0.107195,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,4.0,1.0,182.0,182.000000,6.000000,182.000000,6.000000,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
177163,3164163,1,3864195,49.00,0.021053,1802,555.0,150.0,0.038016,226.0,0.025510,170.0,87.0,116.06686,226.707442,0.045380,0.107195,2.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,4.0,0.0,3.0,1.0,3.0,3.000000,0.000000,2.000000,0.000000,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
177164,3164164,0,3864205,102.00,0.021053,2114,396.0,150.0,0.035411,224.0,0.025510,315.0,87.0,116.06686,226.707442,0.038933,0.107195,13.0,20.0,0.0,0.0,1.0,6.0,0.0,0.0,9.0,0.0,9.0,0.0,53.0,10.0,6.0,6.000000,0.000000,144.315713,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
177165,3164165,1,3864212,24.00,0.021053,10057,225.0,150.0,0.035411,224.0,0.025510,269.0,87.0,116.06686,226.707442,0.038933,0.107195,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,1.0,2.0,2.000000,1.000000,144.315713,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
177166,3164166,0,3864218,422.50,0.021053,16659,170.0,150.0,0.038016,226.0,0.080659,264.0,87.0,116.06686,226.707442,0.045380,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,171.361941,28.115415,0.000000,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
177167,3164167,0,3864222,25.95,0.021053,13052,254.0,150.0,0.038016,226.0,0.025510,325.0,87.0,116.06686,226.707442,0.045380,0.107195,4.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,0.0,3.0,0.0,22.0,4.0,121.0,121.000000,12.000000,158.000000,12.000000,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104

In [21]:
validation.head(10)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceInfo,65,android,chrome,for,generic,mobile,safari,"(desktop,)","(mobile,)","(nan,)"
0,2987000,0,86400,68.5,0.021053,13926,360.037841,150.0,0.099141,142.0,0.080659,315.0,87.0,19.00000,226.707442,0.038933,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,171.361941,13.000000,144.315713,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.076710,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
1,2987001,0,86401,29.0,0.021053,2755,404.000000,150.0,0.035411,102.0,0.080659,325.0,87.0,116.06686,226.707442,0.045380,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,171.361941,28.115415,0.000000,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.076710,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
2,2987002,0,86469,59.0,0.021053,4663,490.000000,150.0,0.038016,166.0,0.025510,330.0,87.0,287.00000,226.707442,0.102670,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,171.361941,28.115415,0.000000,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.076710,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
3,2987003,0,86499,50.0,0.021053,18132,567.000000,150.0,0.035411,117.0,0.025510,476.0,87.0,116.06686,226.707442,0.025593,0.107195,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,112.000000,0.000000,94.000000,0.000000,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.076710,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
4,2987004,0,86506,50.0,0.095829,4497,514.000000,150.0,0.035411,102.0,0.080659,420.0,87.0,116.06686,226.707442,0.045380,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,171.361941,28.115415,144.315713,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,100.000000,0.114087,48.518003,-480.000000,0.071957,0.071221,166.000000,14.379886,542.000000,144.000000,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.072922,0.072281,0.133484,32.000000,0.092105,0.067213,0.075777,0.111923,0.113097,0.080204,0.000000,0,0,0,0,0,0,0,0.0,1.0,0.0
5,2987005,0,86510,49.0,0.021053,5937,555.000000,150.0,0.038016,226.0,0.025510,272.0,87.0,36.00000,226.707442,0.045380,0.107195,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,171.361941,28.115415,0.000000,42.567771,75.469764,37.282984,131.031119,0.553722,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.076710,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0

#Feature Selection

In [22]:
from sklearn.feature_selection import SelectKBest, f_regression

In [23]:
points_train = train.drop(columns=['isFraud'])
labels_train = pd.DataFrame(train['isFraud'])

In [24]:
selector = SelectKBest(f_regression, k=100)

In [25]:
selector.fit(points_train, labels_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SelectKBest(k=100, score_func=<function f_regression at 0x7fafc7ed73b0>)

In [26]:
cols = selector.get_support(indices=True)

In [27]:
cols

array([  3,   6,   9,  15,  22,  27,  48,  68,  69,  70,  71,  74,  75,
        76,  84,  85,  86,  87,  90,  91,  92,  93,  95,  96,  97,  98,
       100, 103, 104, 105, 110, 111, 112, 113, 116, 117, 124, 125, 126,
       127, 130, 131, 132, 133, 134, 137, 138, 139, 140, 145, 146, 147,
       164, 166, 176, 202, 209, 211, 223, 224, 229, 239, 241, 242, 243,
       244, 247, 250, 252, 253, 254, 274, 275, 281, 282, 283, 295, 296,
       297, 298, 299, 300, 301, 302, 305, 307, 310, 311, 312, 315, 335,
       336, 355, 356, 357, 430, 432, 433, 439, 440])

In [28]:
columnas_para_seleccionar = points_train.columns

In [29]:
cont = -1
for c in columnas_para_seleccionar:
  cont += 1
  if cont not in cols and c not in ['id_33', 'DeviceInfo', 'isFraud']:
    train.drop(columns=[c], inplace=True)

In [30]:
train.head()

,isFraud,ProductCD,card3,card6,R_emaildomain,C7,C12,M4,V15,V16,V17,V18,V21,V22,V23,V31,V32,V33,V34,V37,V38,V39,V40,V42,V43,V44,V45,V47,V50,V51,V52,V57,V58,V59,V60,V63,V64,V71,V72,V73,...,V186,V188,V189,V190,V191,V194,V197,V199,V200,V201,V221,V222,V228,V229,V230,V242,V243,V244,V245,V246,V247,V248,V249,V252,V254,V257,V258,V259,V262,V282,V283,V302,V303,V304,id_33,DeviceInfo,android,chrome,"(mobile,)","(nan,)"
177162,0,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.028990,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,0.0,0.0,0.0,0.0,0.0,0.07671,0.104684,0,0,0.0,1.0
177163,1,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.037299,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.000000,2.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,2.0,6.0,0.0,0.0,0.0,0.07671,0.104684,0,0,0.0,1.0
177164,0,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.028990,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.103761,1.158857,0.146788,0.157945,0.13775,0.150655,1.084039,1.120623,1.035898,0.143738,0.158898,0.172663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,2.0,13.0,0.0,0.0,0.0,0.07671,0.104684,0,0,0.0,1.0
177165,1,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.037299,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.103761,1.158857,0.146788,0.157945,0.13775,0.150655,1.084039,1.120623,1.035898,0.143738,0.158898,0.172663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,1.0,3.0,0.0,0.0,0.0,0.07671,0.104684,0,0,0.0,1.0
177166,0,0.021053,150.0,0.080659,0.107195,0.0,0.0,0.037299,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,1.0,1.0,0.0,0.0,0.0,0.07671,0.104684,0,0,0.0,1.0


In [31]:
#aux1 = train['id_33']
#aux2 = train['DeviceInfo']
#aux3 = train['isFraud']
#new_train = train.iloc[:,cols]
##new_train['id_31'] = train['id_31']
#new_train['id_33'] = train['id_33']
##new_train['DeviceType'] = train['DeviceType']
#new_train['DeviceInfo'] = train['DeviceInfo']
#new_train['isFraud'] = train['isFraud']

In [32]:
cont = -1
for c in columnas_para_seleccionar:
  cont += 1
  if cont not in cols and c not in ['id_33', 'DeviceInfo', 'isFraud']:
    validation.drop(columns=[c], inplace=True)

In [33]:
#new_validation = validation.iloc[:,cols]
##new_validation['id_31'] = validation['id_31']
#new_validation['id_33'] = validation['id_33']
##new_validation['DeviceType'] = validation['DeviceType']
#new_validation['DeviceInfo'] = validation['DeviceInfo']
#new_validation['isFraud'] = validation['isFraud']

In [34]:
validation.head()

,isFraud,ProductCD,card3,card6,R_emaildomain,C7,C12,M4,V15,V16,V17,V18,V21,V22,V23,V31,V32,V33,V34,V37,V38,V39,V40,V42,V43,V44,V45,V47,V50,V51,V52,V57,V58,V59,V60,V63,V64,V71,V72,V73,...,V186,V188,V189,V190,V191,V194,V197,V199,V200,V201,V221,V222,V228,V229,V230,V242,V243,V244,V245,V246,V247,V248,V249,V252,V254,V257,V258,V259,V262,V282,V283,V302,V303,V304,id_33,DeviceInfo,android,chrome,"(mobile,)","(nan,)"
0,0,0.021053,150.0,0.080659,0.107195,0.0,0.0,0.123225,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.103761,1.158857,0.146788,0.157945,0.13775,0.150655,1.084039,1.120623,1.035898,0.143738,0.158898,0.172663,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,1.0,1.0,0.0,0.0,0.0,0.076710,0.104684,0,0,0.0,1.0
1,0,0.021053,150.0,0.080659,0.107195,0.0,0.0,0.037299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,1.0,1.0,0.0,0.0,0.0,0.076710,0.104684,0,0,0.0,1.0
2,0,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.037299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,1.0,1.0,0.0,0.0,0.0,0.076710,0.104684,0,0,0.0,1.0
3,0,0.021053,150.0,0.025510,0.107195,0.0,0.0,0.037299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,1.131836,1.030434,1.064345,1.216707,1.035779,0.93518,0.93684,1.285901,1.166581,1.223588,1.283296,1.390153,1.465709,1.715876,1.573082,1.151636,1.199022,1.160753,0.844696,1.252795,1.029979,1.057432,1.039691,1.031605,1.04438,1.342963,1.421111,0.957847,1.009538,0.0,0.0,0.0,0.0,0.0,0.076710,0.104684,0,0,0.0,1.0
4,0,0.095829,150.0,0.080659,0.107195,0.0,0.0,0.051507,0.114853,0.115119,0.125419,0.126085,0.121369,0.123789,1.034559,0.129383,0.130101,0.126064,0.136202,1.103761,1.158857,0.146788,0.157945,0.13775,0.150655,1.084039,1.120623,1.035898,0.143738,0.158898,0.172663,0.11812,0.122731,0.125129,0.133835,0.12177,0.133793,0.128688,0.133586,0.132513,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,0.092105,0.000000,0,0,1.0,0.0


In [35]:
points_train = None
points_label = None
selector = None

#Hiperparámetros

Separo labels y points en train set

In [ ]:
points_train = train.drop(columns=['isFraud'])
labels_train = pd.DataFrame(train['isFraud'])

Utilizo la librería sklearn para hacer un random forest regression con los mejores parámetros posibles

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(random_state=7)

In [ ]:
parametros = [{
    'bootstrap': [True, False],
    'max_depth': [None, 5, 10, 20, 30],
    'max_features': ['auto', 'sqrt', 'log2'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [100, 400, 700, 1000, 1300, 1700, 2000]
}]

In [ ]:
model = RandomizedSearchCV(rf, param_distributions=parametros, n_iter=10, random_state=7, scoring='roc_auc', n_jobs=1, verbose=100, cv=2)

In [ ]:
model.fit(points_train, labels_train)

In [ ]:
model.best_estimator_

#Random Forest Classifier

In [36]:
points_train = train.drop(columns=['isFraud'])
labels_train = pd.DataFrame(train['isFraud'])

In [37]:
from sklearn.ensemble import RandomForestClassifier

In [38]:
rf = RandomForestClassifier(max_features='log2', min_samples_leaf=4,
                       min_samples_split=5, random_state=7)

In [39]:
rf.fit(points_train, labels_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(max_features='log2', min_samples_leaf=4,
                       min_samples_split=5, random_state=7)

In [40]:
from sklearn.metrics import roc_auc_score

In [41]:
points_validation = validation.drop(columns=['isFraud'])
labels_validation = pd.DataFrame(validation['isFraud'])

In [42]:
roc_auc_score(labels_validation, rf.predict_proba(points_validation)[:, 1])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  FutureWarning,


0.8341670641246821

#Importancia features

In [ ]:
feature_importance = pd.DataFrame(rf.feature_importances_, index=points_train.columns, columns=['importance']).sort_values('importance', ascending=False)

In [ ]:
feature_importance.head(10)

In [ ]:
feature_importance.head(10)['importance'].plot(kind='bar')
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Los 10 features más importantes')

In [ ]:
feature_importance = None

#Matriz de confusión

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

In [ ]:
conf_matrix = confusion_matrix(labels_validation, rf.predict(points_validation))
conf_matrix

In [ ]:
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot()

In [ ]:
conf_matrix = None
disp = None

#Limpio la RAM

In [43]:
train = None
validation = None
points_train = None
points_validation = None
labels_train = None
labels_validation = None

#Test set

In [44]:
id = '11-pP9l-0FTW9A1pwrb2nVeAoV8Awf4oY'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_transaction.csv')

test_transaction = pd.read_csv('test_transaction.csv')

In [45]:
id = '1xHLcsTe2619N38GI9O9QzPV74XRrUKbx'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('test_identity.csv')

test_identity = pd.read_csv('test_identity.csv')

In [46]:
downloaded = None

Mergeo los dos datasets que tengo

In [47]:
df_test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')

In [48]:
test_transaction = None
test_identity = None

In [49]:
df_test.head(10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id-01,id-02,id-03,id-04,id-05,id-06,id-07,id-08,id-09,id-10,id-11,id-12,id-13,id-14,id-15,id-16,id-17,id-18,id-19,id-20,id-21,id-22,id-23,id-24,id-25,id-26,id-27,id-28,id-29,id-30,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,472.0,87.0,2635.0,NaN,hotmail.com,NaN,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,NaN,NaN,NaN,NaN,136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,205.0,87.0,17.0,NaN,gmail.com,NaN,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,NaN,NaN,NaN,NaN,242.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,264.0,87.0,6.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,NaN,NaN,NaN,NaN,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3663554,18403323,57.95,W,12839,321.0,150.0,visa,226.0,debit,512.0,87.0,NaN,NaN,gmail.com,NaN,5.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,0.0,4.0,0.0,10.0,4.0,36.0,36.0,35.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,3663555,18403350,87.00,W,16560,476.0,150.0,visa,126.0,debit,110.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3663556,18403387,390.00,W,15066,170.0,150.0,mastercard,102.0,credit,194.0,87.0,303.0,NaN,gmail.com,NaN,3.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,0.0,1.0,1.0,11.0,1.0,NaN,NaN,NaN,126.0,4.0,NaN,NaN,NaN,NaN,126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3663557,18403405,103.95,W,2803,100.0,150.0,visa,226.0,debit,494.0,87.0,3.0,NaN,gmail.com,NaN,152.0,148.0,0.0,0.0,135.0,95.0,0.0,0.0,77.0,0.0,122.0,0.0,407.0,108.0,128.0,128.0,13.0,644.0,13.0,NaN,NaN,NaN,NaN,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3663558,18403416,117.00,W,12544,321.0,150.0,visa,226.0,debit,476.0,87.0,8.0,NaN,NaN,NaN,2.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,2.0,1.0,8.0,2.0,69.0,69.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,68.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [50]:
ids = df_test[['TransactionID']]

#Me aseguro que las columnas de test sean las mismas que las de train

In [51]:
columnas = columnas_train.drop('isFraud')

In [52]:
df_test.columns = columnas

In [53]:
df_test.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [54]:
columnas = None

#Encoding test set

In [55]:
for c in df_test.columns:
    print(f"columna {c}")
    if c == 'id_31':
      vectorizer = dic[c][0]
      df_test[c].fillna(' ', inplace=True)

      feature = vectorizer.transform(df_test.id_31).toarray()
      new_df = pd.DataFrame(feature, columns=vectorizer.get_feature_names(), index=df_test.index)
      df_test.drop(columns=['id_31'], inplace=True)
      for i in new_df.columns:
        df_test[i] = new_df[i]

      vectorizer = None
      feature = None
      feature_labels = None
      new_df = None

      continue

    if c == 'DeviceType':
      ohe = dic[c][0]

      feature = ohe.transform(df_test[[c]]).toarray()
      new_df = pd.DataFrame(feature, columns=ohe.categories_, index=df_test.index)
      df_test.drop(columns=['DeviceType'], inplace=True)
      for i in new_df.columns:
        df_test[i] = new_df[i]

      ohe = None
      feature = None
      feature_labels = None
      new_df = None

      continue

    if (df_test[c].dtype.kind not in 'biufc'): #si no es numérica la columna
      mean_encoded_subject = dic[c][0]

      df_test[c] = df_test[c].map(mean_encoded_subject)

      mean_encoded_subject = None

    aux = dic[c][1]
    if aux:
      df_test[c].fillna(aux, inplace=True)

columna TransactionID
columna TransactionDT
columna TransactionAmt
columna ProductCD
columna card1
columna card2
columna card3
columna card4
columna card5
columna card6
columna addr1
columna addr2
columna dist1
columna dist2
columna P_emaildomain
columna R_emaildomain
columna C1
columna C2
columna C3
columna C4
columna C5
columna C6
columna C7
columna C8
columna C9
columna C10
columna C11
columna C12
columna C13
columna C14
columna D1
columna D2
columna D3
columna D4
columna D5
columna D6
columna D7
columna D8
columna D9
columna D10
columna D11
columna D12
columna D13
columna D14
columna D15
columna M1
columna M2
columna M3
columna M4
columna M5
columna M6
columna M7
columna M8
columna M9
columna V1
columna V2
columna V3
columna V4
columna V5
columna V6
columna V7
columna V8
columna V9
columna V10
columna V11
columna V12
columna V13
columna V14
columna V15
columna V16
columna V17
columna V18
columna V19
columna V20
columna V21
columna V22
columna V23
columna V24
columna V25
columna V26

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


columna id_32
columna id_33
columna id_34
columna id_35
columna id_36
columna id_37
columna id_38
columna DeviceType
columna DeviceInfo


In [56]:
df_test.head(10)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceInfo,65,android,chrome,for,generic,mobile,safari,"(desktop,)","(mobile,)","(nan,)"
0,3663549,18403224,31.95,0.021053,10409,111.0,150.0,0.038016,226.0,0.025510,170.0,87.0,1.00000,226.707442,0.045380,0.107195,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.000000,419.000000,27.000000,398.000000,27.000000,75.469764,37.282984,131.031119,0.553722,418.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
1,3663550,18403263,49.00,0.021053,4272,111.0,150.0,0.038016,226.0,0.025510,299.0,87.0,4.00000,226.707442,0.024539,0.107195,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.000000,149.000000,7.000000,634.000000,7.000000,75.469764,37.282984,131.031119,0.553722,231.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
2,3663551,18403310,171.00,0.021053,4476,574.0,150.0,0.038016,226.0,0.025510,472.0,87.0,2635.00000,226.707442,0.055550,0.107195,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.000000,137.000000,10.000000,97.000000,10.000000,75.469764,37.282984,131.031119,0.553722,136.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
3,3663552,18403310,284.95,0.021053,10989,360.0,150.0,0.038016,166.0,0.025510,205.0,87.0,17.00000,226.707442,0.045380,0.107195,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.000000,42.000000,41.000000,242.000000,41.000000,75.469764,37.282984,131.031119,0.553722,242.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
4,3663553,18403317,67.95,0.021053,18018,452.0,150.0,0.035411,117.0,0.025510,264.0,87.0,6.00000,226.707442,0.045380,0.107195,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.000000,22.000000,0.000000,22.000000,0.000000,75.469764,37.282984,131.031119,0.553722,22.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.109691,0.109691,0.109691,0.104684,0,0,0,0,0,0,0,0.0,0.0,1.0
5,3663554,18403323,57.95,0.021053,12839,321.0,150.0,0.038016,226.0,0.025510,512.0,87.0,116.06686,226.707442,0.045380,0.107195,5.0,5.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,0.0,4.0,0.0,10.0,4.0,36.000000,36.000000,35.000000,0.000000,42.567771,75.469764,37.282984,131.031119,0.553722,0.0,...,0.070348,-0.274654,99.773942,0.108306,48.518003,-339.278514,0.109691,0.110087,196.754465,14.379886,349.094953,416.232534,367.312224,16.509919,0.1205,12.635556,332.540971,148.895105,0.1205,0.109648,0.109648,0.075285,26.216168,0.07671,0.075925,0.109691,0.10

#Elimino columnas test set

In [57]:
l = []

In [58]:
cont = -1
for c in columnas_para_seleccionar:
  cont += 1
  if cont not in cols and c not in ['id_33', 'DeviceInfo', 'isFraud']:
    l.append(c)

In [59]:
df_test.drop(columns=l, inplace=True)

#Predicciones

Este fue el mejor modelo así que lo envío a la competencia

In [62]:
df_predictions = pd.DataFrame(list(rf.predict_proba(df_test)[:, 1]))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['str', 'tuple']. An error will be raised in 1.2.
  FutureWarning,


In [63]:
df_predictions['TransactionID'] = ids

In [64]:
df_predictions.columns = ['isFraud', 'TransactionID']

In [65]:
df_predictions.head(10)

,isFraud,TransactionID
0,0.009437,3663549
1,0.019157,3663550
2,0.018223,3663551
3,0.010080,3663552
4,0.006855,3663553
5,0.006855,3663554
6,0.017805,3663555
7,0.063277,3663556
8,0.006084,3663557
9,0.023539,3663558


In [66]:
df_predictions = df_predictions[['TransactionID', 'isFraud']]

In [67]:
df_predictions.head(10)

,TransactionID,isFraud
0,3663549,0.009437
1,3663550,0.019157
2,3663551,0.018223
3,3663552,0.010080
4,3663553,0.006855
5,3663554,0.006855
6,3663555,0.017805
7,3663556,0.063277
8,3663557,0.006084
9,3663558,0.023539


In [68]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [69]:
df_predictions.to_csv('RF_N10_C2.csv', index=False)
!cp RF_N10_C2.csv "drive/My Drive/"

El score obtenido en la competencia es 0.871224